In [61]:
import pandas as pd, numpy as np
import tensorflow as tf
import copy, time, string
from sklearn import svm
from sklearn.decomposition import PCA
import xgboost as xgb

from Tools import *

In [62]:
from imp import reload
reload(Tools)
reload(tensorflow)

NameError: name 'Tools' is not defined

In [63]:
dfTrainOri = pd.read_csv(filepath_or_buffer = 'data/train.csv')
dfTestOri = pd.read_csv(filepath_or_buffer = 'data/test.csv')
dfTestYOri = pd.read_csv(filepath_or_buffer = 'data/gender_submission.csv')

In [64]:
dfWhole = pd.concat([dfTrainOri, dfTestOri])
dfWhole

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450
5,NaN,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877
6,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,male,0,0.0,17463
7,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,male,3,0.0,349909
8,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,female,0,1.0,347742
9,14.0,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,female,1,1.0,237736


In [44]:
# /********************** 加入同票组内平均幸存率指标 ******************/
dfWhole.index = np.arange(dfWhole.shape[0])
grpsByTicketWhole = dfWhole.groupby('Ticket').groups
for ticket in grpsByTicketWhole:
    if grpsByTicketWhole[ticket].shape == (1,):
        surviveRateAverage = np.nan
    else:
        surviveRateAverage = np.mean(dfWhole.loc[grpsByTicketWhole[ticket], 'Survived'])
    dfWhole.loc[grpsByTicketWhole[ticket], 'IngroupSurviveRateOfTicket'] = surviveRateAverage
dfWhole.loc[pd.isnull(dfWhole['IngroupSurviveRateOfTicket']), 'IngroupSurviveRateOfTicket'] = np.mean(dfWhole['IngroupSurviveRateOfTicket'])

In [173]:
dfWhole

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,IngroupSurviveRateOfTicket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,0.522623
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,1.000000
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,0.522623
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,0.500000
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,0.522623
5,NaN,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877,0.522623
6,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,male,0,0.0,17463,0.000000
7,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,male,3,0.0,349909,0.000000
8,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,female,0,1.0,347742,1.000000
9,14.0,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,female,1,1.0,237736,0.500000


In [70]:
dfWhole.to_csv(path_or_buf='data\\dfWhole2.csv')

In [71]:
dfWhole.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived,IngroupSurviveRateOfTicket
count,1046.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000,1309.000000
mean,29.881138,33.295479,0.385027,655.000000,2.294882,0.498854,0.383838,0.522623
std,14.413493,51.758668,0.865560,378.020061,0.837836,1.041658,0.486592,0.283549
min,0.170000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,21.000000,7.895800,0.000000,328.000000,2.000000,0.000000,0.000000,0.522623
50%,28.000000,14.454200,0.000000,655.000000,3.000000,0.000000,0.000000,0.522623
75%,39.000000,31.275000,0.000000,982.000000,3.000000,1.000000,1.000000,0.522623
max,80.000000,512.329200,9.000000,1309.000000,3.000000,8.000000,1.000000,1.000000


In [65]:
dfWhole.index = dfWhole['PassengerId']
dfWholeY = dfWhole['Survived']
dfWholeX = copy.copy(dfWhole)
dfWholeX['SEmbarked'] = dfWholeX['Embarked'] == 'S'
dfWholeX['CEmbarked'] = dfWholeX['Embarked'] == 'C'
dfWholeX['QEmbarked'] = dfWholeX['Embarked'] == 'Q'
dfWholeX['Class1'] = dfWholeX['Pclass'] == 1
dfWholeX['Class2'] = dfWholeX['Pclass'] == 2
dfWholeX['Class3'] = dfWholeX['Pclass'] == 3
dfWholeX['nameLength'] = dfWholeX['Name'].apply(len)
dfWholeX.loc[dfWholeX['Sex'] == 'male', 'Sex'] = 1
dfWholeX.loc[dfWholeX['Sex'] == 'female', 'Sex'] = 0

dfWholeX = dfWholeX.drop(['PassengerId','Survived','Name','Ticket','Cabin','Embarked','Pclass'], axis = 1)
dfWholeX.loc[np.isnan(dfWholeX['Age']), 'Age'] = np.mean(dfWholeX['Age'])

MeanFareClass1 = np.mean(dfWholeX.loc[dfWholeX['Class1'], 'Fare'])
MeanFareClass2 = np.mean(dfWholeX.loc[dfWholeX['Class2'], 'Fare'])
MeanFareClass3 = np.mean(dfWholeX.loc[dfWholeX['Class3'], 'Fare'])
MeanFare = [MeanFareClass1, MeanFareClass2, MeanFareClass3]
FareIsNullIds = dfWholeX.index[pd.isnull(dfWholeX['Fare'])]
for fareIsNullId in FareIsNullIds:
    dfWholeX.loc[fareIsNullId, 'Fare'] = np.sum(dfWholeX.loc[fareIsNullId, ['Class1', 'Class2', 'Class3']] * MeanFare)
    print('PassengerId %d has renewed its "Fare" into %f'%(fareIsNullId, dfWholeX.loc[fareIsNullId, 'Fare']))

PassengerId 1044 has renewed its "Fare" into 13.302889


In [66]:
NontestIndex = ~pd.isnull(dfWhole['Survived'])
NontestCount = np.sum(NontestIndex)
dfNontestX = dfWholeX.loc[NontestIndex, :]
dfNontestY = dfWholeY.loc[NontestIndex]
dfTestX = dfWholeX.loc[~NontestIndex, :]
dfTestY = dfWholeY.loc[~NontestIndex]
trainCount = int(NontestCount*0.8)
devCount = NontestCount - trainCount
print("训练集大小：%d\t开发集大小：%d\t测试集大小：%d"%(trainCount, devCount, dfWhole.shape[0] - NontestCount))

permut = np.random.permutation(NontestCount)
dfTrainX = dfWholeX.iloc[permut[:trainCount], :]
dfTrainY = dfWholeY.iloc[permut[:trainCount]]
dfDevX = dfWholeX.iloc[permut[trainCount:NontestCount], :]
dfDevY = dfWholeY.iloc[permut[trainCount:NontestCount]]

训练集大小：712	开发集大小：179	测试集大小：418


In [67]:
arTrainX = np.array(dfTrainX).T
arTrainY = np.array(dfTrainY).T.reshape([1, -1])
arDevX = np.array(dfDevX).T
arDevY = np.array(dfDevY).T.reshape([1, -1])
arTestX = np.array(dfTestX).T
arTestY = np.array(dfTestY).T.reshape([1, -1])

In [68]:
dpTrainX = dataPool(data = arTrainX, axis = 1)
dpTrainY = dataPool(data = arTrainY.reshape([1,-1]), axis = 1)
dpTestX = dataPool(data = arTestX, axis = 1)
dpTestY = dataPool(data = arTestY.reshape([1,-1]), axis = 1)

In [69]:
Xrange = [np.min(arTrainX, axis = 1).reshape([-1, 1]), np.max(arTrainX, axis = 1).reshape([-1, 1])]
arTrainX =  (arTrainX - Xrange[0])/ (Xrange[1] - Xrange[0]) * 2 - 1
arDevX = (arDevX - Xrange[0]) / (Xrange[1] - Xrange[0]) * 2 - 1
arTestX = (arTestX - Xrange[0]) / (Xrange[1] - Xrange[0] * 2) - 1

pca = PCA(n_components = 5)
#pca.fit(X = arTrainX.T, y = arTrainY.T)
pca.fit(X = arTrainX.T)
print("explained_variance_ratio_:", pca.explained_variance_ratio_)
arTrainX = pca.transform(arTrainX.T).T
arDevX = pca.transform(arDevX.T).T
arTestX = pca.transform(arTestX.T).T
print("sum of above:", sum(pca.explained_variance_ratio_))


explained_variance_ratio_: [ 0.3091218   0.24800822  0.16669216  0.14150353  0.07260682]
sum of above: 0.93793252555


In [51]:
arTestX.shape

(5, 418)

In [123]:
layersize = [20, 20, 20, 1]
layercount = len(layersize)

seed = time.time()
X = tf.placeholder(shape = [arTrainX.shape[0], None], dtype = 'float')
Y_ = tf.placeholder(shape = [layersize[-1], None], dtype = 'float')
W = dict()
b = dict()
Y = dict()
Y[0] = X
layersizeTemp = [arTrainX.shape[0]] + layersize
for i in range(layercount):
    W[i+1] = tf.Variable(tf.random_normal(seed = np.random.randint(seed), shape = [layersizeTemp[i+1], layersizeTemp[i]]) * 0.1)
    b[i+1] = tf.Variable(tf.zeros(shape = [layersizeTemp[i+1], 1]))
    if i != layercount - 1:
        Y[i+1] = tf.nn.sigmoid(tf.matmul(W[i+1], Y[i]) + b[i+1])
Y[layercount] = tf.nn.sigmoid(tf.matmul(W[layercount], Y[layercount-1]) + b[layercount])
Youtput = Y[layercount]

correct_prediction = tf.equal(tf.round(Youtput), Y_)
correct_rate = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

cost = tf.reduce_mean(tf.square(Youtput - Y_))
#for i in range(len(layersize)):
#    cost += 0.000005 * tf.reduce_sum(tf.square(W[i+1]))

#cost = - tf.reduce_mean(Y_*tf.log(Youtput) + (1 - Y_)*tf.log(1 - Youtput))

In [125]:
#train_step = tf.train.AdagradOptimizer(1).minimize(crossEntropy)
optimizer = tf.train.AdamOptimizer(0.1).minimize(cost)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init) 

for i in range(10001):
    #sess.run(train_step, feed_dict = {X:dpTrainX.getData(size=50), Y_:dpTrainY.getData(size=50)})
    sess.run(optimizer, feed_dict = {X:arTrainX, Y_:arTrainY})
    if i % 1000 == 0:
        predTrain = sess.run(Youtput, feed_dict={X:arTrainX, Y_: arTrainY})
        predDev = sess.run(Youtput, feed_dict={X:arDevX, Y_: arDevY})
        print(calculateAuc(arTrainY.reshape([-1]), predTrain.reshape([-1])), calculateAuc(arDevY.reshape([-1]), predDev.reshape([-1])), 
             np.mean(np.round(predTrain).reshape([-1]) == arTrainY.reshape([-1])), np.mean(np.round(predDev).reshape([-1]) == arDevY.reshape([-1])))

0.839167706261 0.826392437404 0.626404494382 0.575418994413
0.882055194039 0.828564128769 0.827247191011 0.759776536313
0.878514953303 0.835015329586 0.844101123596 0.776536312849
0.879467446644 0.832971384773 0.835674157303 0.782122905028
0.841949323983 0.827925396014 0.830056179775 0.810055865922
0.864227553188 0.827414409811 0.832865168539 0.787709497207
0.862558582555 0.829011241696 0.828651685393 0.776536312849
0.854989210695 0.826775677057 0.825842696629 0.798882681564
0.863586938198 0.819877363311 0.834269662921 0.793296089385
0.85774132641 0.807677567706 0.839887640449 0.787709497207
0.856788833069 0.80077925396 0.837078651685 0.787709497207


,0,1,2,3,4,5,6,7,8,9,...,169,170,171,172,173,174,175,176,177,178
0,-1.397491,-0.171227,-0.188624,-0.444993,-1.390343,-1.371750,-0.871277,-0.184508,-0.212615,0.903412,...,2.213728,2.771124,0.996334,-1.359318,-1.374960,2.778894,-1.384606,-1.376634,1.564431,-0.843599
1,-0.037814,2.013682,2.011399,1.765580,-0.040437,-0.046395,0.069578,2.014546,2.023776,-1.660636,...,1.442275,1.541371,-0.620979,-0.053268,-0.047389,1.541971,-0.040627,-0.061015,-0.486987,0.072603
2,-0.499318,-0.569835,-0.485610,1.724392,-0.500755,-0.508093,1.399254,-0.496731,-0.511538,1.458233,...,-1.440319,0.440587,-1.408312,-0.477474,-0.481238,0.404106,-0.522909,-0.324501,0.351707,1.294757
3,-0.140502,1.147576,1.116608,0.224645,-0.145901,-0.158660,-0.961346,1.130682,1.160044,0.503543,...,0.275821,-0.473369,-0.993780,-0.180652,-0.166161,-0.471628,-0.142306,-0.237758,-1.715628,-0.859816
4,-0.204709,-0.917404,-0.996006,1.495944,-0.199246,-0.183234,-0.463122,-0.982431,-0.984564,0.012299,...,-0.248960,-0.466704,0.515559,-0.198893,-0.205078,-0.427996,-0.179589,-0.328711,0.357755,-0.356796


In [82]:
sess.run(Youtput, feed_dict = {X:arDevX})

array([[ 0.10695942,  0.21935259,  0.28344828,  0.73855382,  0.10804545,
         0.10976163,  0.38522285,  0.27154544,  0.24669003,  0.90460759,
         0.92620623,  0.10429811,  0.6133092 ,  0.27124748,  0.14628649,
         0.90191376,  0.42193177,  0.38383296,  0.94918001,  0.73966497,
         0.20681867,  0.73829573,  0.39364648,  0.0707997 ,  0.94978476,
         0.11022006,  0.10595542,  0.10715074,  0.90521067,  0.23875485,
         0.90162903,  0.11993281,  0.92973679,  0.15990259,  0.92449552,
         0.21973674,  0.92786843,  0.09687792,  0.12651783,  0.40571076,
         0.10830044,  0.59868342,  0.30801758,  0.36914819,  0.11486257,
         0.92604274,  0.40215278,  0.95054185,  0.59349477,  0.12811252,
         0.12658541,  0.11659276,  0.36738726,  0.10981844,  0.11484522,
         0.4172726 ,  0.26939234,  0.09817507,  0.27072445,  0.73176378,
         0.13265279,  0.92750996,  0.13741106,  0.89916933,  0.10614357,
         0.90165901,  0.27450529,  0.9490301 ,  0.3

In [126]:
preds = sess.run(Youtput, feed_dict = {X:arTestX})

In [127]:
np.round(preds)

array([[ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,
         1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,
         1.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,
         0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,
         1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
         1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,
         0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,
         1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,
         0.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,
         1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.

In [112]:
sess.run(W[3], feed_dict = {X:arTestX})

array([[ -1.01077566e+01,   9.46740270e-01,  -1.78045311e+01,
         -2.73308620e+01,  -9.71669292e+00,   1.26132345e+01,
          1.70570850e+00,   7.00395441e+00,  -1.49283705e+01,
          3.09586620e+00,   7.57942915e+00,   4.30115080e+00,
         -6.26424456e+00,  -4.46547794e+00,  -3.12892771e+00,
          8.71313095e+00,  -1.67410069e+01,   2.75644094e-01,
          2.45336628e+01,  -4.14597178e+00],
       [ -3.63541508e+00,  -1.37225461e+00,  -9.98548794e+00,
         -1.30063117e+00,  -2.99828506e+00,  -5.35383940e-01,
         -4.48761559e+00,  -5.13070524e-01,  -9.49632931e+00,
         -1.61004257e+00,   4.53697968e+00,  -1.16448939e+00,
         -2.13444328e+00,  -7.06683934e-01,  -1.26306504e-01,
         -3.19203091e+00,   1.73965931e+00,  -2.56276464e+00,
         -3.02578187e+00,  -1.90225947e+00],
       [ -7.87189305e-02,  -1.21818466e+01,  -1.24029732e+01,
         -3.44097644e-01,  -2.35528803e+00,   7.78820944e+00,
          3.34202719e+00,  -1.21917162e+01

In [44]:
np.mean(preds.reshape([1, -1]) == arTestY)

0.0

### SVC        0.9618  0.6364

In [89]:
clf = svm.SVC(kernel = 'rbf', gamma = 100, C = 50)
clf.fit(arTrainX.T, arTrainY.reshape([-1]))
predTrain = clf.predict(arTrainX.T)
precisionTrain = sum(predTrain == arTrainY.reshape([-1])) / predTrain.shape[0]
print(precisionTrain)
predDev = clf.predict(arDevX.T)
precisionDev = sum(predDev == arDevY.reshape([-1])) / predDev.shape[0]
print(precisionDev)
pred = clf.predict(arTestX.T)

0.898876404494
0.748603351955


,0,1,2,3,4,5,6,7,8,9,...,408,409,410,411,412,413,414,415,416,417
0,-0.012344,0.077109,0.883703,-0.216885,0.075755,-0.212879,0.245101,0.660727,0.655866,-0.214064,...,0.261656,0.030853,0.248044,1.478422,0.060327,-0.211470,1.846269,-0.189031,-0.209957,0.386168
1,1.123315,0.324545,0.247384,0.277281,0.318796,0.272408,1.185718,-0.596467,1.357953,0.273099,...,1.177179,0.337880,1.184190,0.881434,0.330646,0.275061,1.082604,0.265324,0.274282,1.302082
2,0.529935,1.244049,0.623685,0.333015,1.317591,0.377093,1.424651,0.481677,1.277239,0.371175,...,1.458194,1.279846,1.430848,1.054192,1.252342,0.336079,0.763904,0.357404,0.339118,0.351117
3,0.579510,-0.353906,1.237852,0.030030,-0.382675,0.009963,0.237012,0.676048,0.297611,0.003552,...,0.210462,-0.330870,0.232203,-0.257697,-0.338304,0.024352,-0.102558,-0.002469,0.021936,0.655391
4,1.004941,-0.013720,1.193970,0.062017,-0.072791,0.031390,0.919339,0.195300,-0.441888,0.035028,...,0.903813,-0.070574,0.916392,1.227046,-0.030515,0.062913,-0.055636,0.059978,0.061516,-0.343708


### XGboost

In [50]:
dtrain = xgb.DMatrix(arTrainX.T, label=arTrainY.reshape([-1]))

In [51]:
xgb.train(params = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }, dtrain = dtrain)

In [52]:
bst = xgb.Booster()

In [53]:
bst.predict(data = arTrainX.T)

AttributeError: 'numpy.ndarray' object has no attribute 'feature_names'

In [301]:
predTrain = bst.predict(data = arTrainX.T)
precisionTrain = np.sum(predTrain == arTrainY.reshape([-1])) / predTrain.shape[0]
print(precisionTrain)

AttributeError: 'numpy.ndarray' object has no attribute 'feature_names'

In [ ]:
predTest = bst.predict(arTestX.T, ntree_limit=bst.best_iteration)
precisionTest = sum(predTest == arTestY.reshape([-1])) / predTest.shape[0]
print(precisionTest)

In [287]:
help(bst.predict)

Help on method predict in module xgboost.core:

predict(data, output_margin=False, ntree_limit=0, pred_leaf=False) method of xgboost.core.Booster instance
    Predict with data.
    
    NOTE: This function is not thread safe.
          For each booster object, predict can only be called from one thread.
          If you want to run prediction using multiple thread, call bst.copy() to make copies
          of model object and then call predict
    
    Parameters
    ----------
    data : DMatrix
        The dmatrix storing the input.
    
    output_margin : bool
        Whether to output the raw untransformed margin value.
    
    ntree_limit : int
        Limit number of trees in the prediction; defaults to 0 (use all trees).
    
    pred_leaf : bool
        When this option is on, the output will be a matrix of (nsample, ntrees)
        with each record indicating the predicted leaf index of each sample in each tree.
        Note that the leaf index of a tree is unique per tree, 

In [28]:
preds.shape

(1, 179)

In [26]:
result.shape

(418, 1)

In [128]:
result = pd.DataFrame({'PassengerId':dfTestYOri['PassengerId']} )
result['Survived'] = list(map(lambda x:int(round(x)), preds.reshape([-1])))

In [129]:
result

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
5,897,1
6,898,1
7,899,1
8,900,1
9,901,0


In [130]:
np.sum(result['Survived']==1)

191

In [131]:
result
result.to_csv(path_or_buf='data\\result20171007.csv', header=True, index=None)

In [122]:
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,1
6,898,1
7,899,1
8,900,0
9,901,1
